## Solution: Setting up the Agents

## Install the packages

In [ ]:
%pip install azure-ai-projects==1.0.0b12
%pip install azure-identity
%pip install azure-ai-agents==1.1.0b3

## Import the libraries

In [2]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import BingGroundingTool, MessageRole

## Create the AI Project Client

NOTE: 
- If you don't have Azure CLI, you first need to install it:
   curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash
- Login to Azure
   az login



In [3]:
# Create an Azure AI Client from an endpoint, copied from your Azure AI Foundry project.
# You need to login to Azure subscription via Azure CLI and set the environment variables
project_endpoint = os.environ["PROJECT_ENDPOINT"]  # Ensure the PROJECT_ENDPOINT environment variable is set

# Create an AIProjectClient instance
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),  # Use Azure Default Credential for authentication
)

## Run Agent Helper Function


In [4]:
def run_agent(user_input, thread_id, agent_id):
    # Add a message to the thread
    message = project_client.agents.messages.create(
        thread_id=thread_id,  # ID of the thread to which the message belongs
        role="user",  # Role of the message sender
        content=user_input,  # Message content
    )
    print(f"Created message, ID: {message['id']}")

     # Create and process agent run in thread with tools
    run = project_client.agents.runs.create_and_process(thread_id=thread_id, agent_id=agent_id)
    print(f"Run finished with status: {run.status}")

    # Check the status of the run and print the result
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    elif run.status == "completed":
        last_msg = project_client.agents.messages.get_last_message_text_by_role(thread_id=thread_id, role=MessageRole.AGENT)
        if last_msg:
            print(f"Agent Response: {last_msg.text.value}")

## Create Weather Agent

In [ ]:
# Enter Bing connection names here
bing_search = "bingresource"  # Replace with your Bing connection name

# Extract the connection list.
conn_list = project_client.connections.list()
bing_search_conn_id = ""

# Find the connection IDs for Bing Search and Bing Custom Search
for conn in conn_list:
    if conn.name == bing_search:
        bing_search_conn_id = conn.id

# Print the connection names
print(f"Bing Search Connection ID: {bing_search_conn_id}")

# Initialize the Bing Grounding tools
bing_search = BingGroundingTool(connection_id=bing_search_conn_id)

# Create an agent with the Bing Grounding tool
agent = project_client.agents.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],  # Model deployment name
    name="weather-agent",  # Name of the agent
    instructions="You are a helpful agent that provides weather information",  # Instructions for the agent
    tools=bing_search.definitions  # Attach the Bing Grounding tool
)

print(f"Created agent, ID: {agent.id}")

# Create a thread for communication
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

# Example user input for the agent
user_input = "What is the weather in Paris?"  # Example user input
run_agent(user_input, thread_id=thread.id, agent_id=agent.id)  # Run the agent with the user input


Bing Search Connection ID: /subscriptions/73047bc3-f020-4930-b16d-fe5c9dd3c73e/resourceGroups/agent_challenge_resource_group/providers/Microsoft.CognitiveServices/accounts/foundrygo2u/projects/agent_challenge_projectgo2u/connections/bingresource
Created agent, ID: asst_0NHYVkCkHZsP66inhvkiqehu
Created thread, ID: thread_UVBLFp328zH3U0KGolsYVtaz


## Create Itinerary Agent

In [14]:
# Create an agent with the Bing Grounding tool
agent = project_client.agents.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],  # Model deployment name
    name="itinerary-agent",  # Name of the agent
    instructions="You are a helpful agent that creates an itinerary given the cities' weather information.",  # Instructions for the agent
)

print(f"Created agent, ID: {agent.id}")

# Create a thread for communication
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

user_input = """Create an itinerary for Paris given this weather information:
Currently, in Paris, it is partly cloudy with a temperature of 68°F (20°C). 
The forecast for later today predicts heavy downpours and frequent lightning with afternoon thundershowers. 
The high temperature will reach 69°F (21°C), with a 100% chance of rain
"""  # Example user input

run_agent(user_input, thread_id=thread.id, agent_id=agent.id)  # Run the agent with the user input

Created agent, ID: asst_hxizwraO7QEbWOGOYh9wicYS
Created thread, ID: thread_sE6OFlCCAFwuKomqb6pRlMbM
Created message, ID: msg_qaVOjDXCElEyrM0MXAuQxo9X
Run finished with status: completed
Agent Response: Sure! Given the weather conditions in Paris today, it's best to plan indoor activities to stay dry and safe. Here's an itinerary designed to make the most of your day despite the rain:

### Morning:

**9:00 AM - 10:30 AM: Breakfast at a Parisian Café**
- Start your day with a classic French breakfast. Head to a nearby café, such as Café de Flore or Les Deux Magots, and enjoy coffee, croissants, and maybe some fresh fruit.

**10:30 AM - 12:30 PM: Visit the Louvre Museum**
- Spend the morning exploring one of the world's largest and most famous museums. You can admire masterpieces such as the Mona Lisa, the Venus de Milo, and many other incredible pieces of art.

### Afternoon:

**12:30 PM - 2:00 PM: Lunch at a Traditional French Restaurant**
- After the museum, head to a traditional Fren

## Travel Product Agent

## Restaurant Agent